In [ ]:
#%%
import numpy as np
import qiskit_metal as metal
from qiskit_metal import designs
from qiskit_metal import MetalGUI, Dict, open_docs
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.framed_path import RouteFramed
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee


from qiskit_metal.analyses import EPRanalysis

design = designs.DesignPlanar()
design.overwrite_enabled = True

design.chips.main.size['size_x'] = '7mm'
design.chips.main.size['size_y'] = '7mm'
design.chips.main.size['size_z'] = '-280um'

gui = MetalGUI(design)

RES_WIDTH = '20um'
RES_GAP = '20um'
LINE_50_OHM_WIDTH = '15um'
LINE_50_OHM_GAP = '10um'

In [ ]:
"""ESTIMATE LENGTH"""
eps_eff = 6.44 # assuming eps_r = 11.9
vp = 3e8/np.sqrt(eps_eff)

def freq2length(f, vp=vp):
    return int(vp/f*1e6/4) #length of lbda/4 resonator

def length2freq(l, vp=vp):
    return int(vp/4/(l*1e-6))

l = freq2length(5e9)
print(l)
f = print(length2freq(l))

In [ ]:
"""CREATE RESONATORS"""

def resonator(ID, f, tee_pos, coupling_length, prime_length, coupling_space, short_pos, res_gap, down_length = 100, fillet = 60,  right = False, orientation = '90'):
    
    if right:
        mirror = False
    else:
        mirror = True
    tee_options = Dict(prime_width=LINE_50_OHM_WIDTH, prime_gap=LINE_50_OHM_GAP, prime_length_1=str(prime_length)+'um', prime_length_2=str(prime_length)+'um',
                    second_width=RES_WIDTH, second_gap=res_gap, coupling_space=str(coupling_space)+'um', coupling_length=str(coupling_length)+'um', 
                    down_length=str(down_length)+'um', fillet=str(fillet)+'um', mirror=mirror, open_termination=True,
                    pos_x=str(tee_pos[0])+'um', pos_y=str(tee_pos[1])+'um', orientation=orientation)
    tee1 = CoupledLineTee(design, 'tee'+ID, options=tee_options)

    spacing = 200
    L = int(freq2length(f)) - coupling_length - down_length

    short1_options = Dict(pos_x=str(short_pos[0])+'um', pos_y=str(short_pos[1])+'um', orientation='-90', termination_gap='10um')
    short1 = ShortToGround(design,'short'+ID,options=short1_options)

    #f = 5e9
    res1_options = Dict(total_length=str(L)+'um', fillet=str(-5+spacing/2)+'um', lead = dict(start_straight='100um', end_straight='0um'),
                        pin_inputs=Dict(start_pin=Dict(component='tee'+ID, pin='second_end'),
                                        end_pin=Dict(component='short'+ID, pin='short')),
                        meander=Dict(spacing=str(spacing)+'um', asymmetry='0um'),
                        trace_width=RES_WIDTH, trace_gap=res_gap)
    res1 = RouteMeander(design, 'res'+ID,  options=res1_options)

In [ ]:
def render_qiskit_metal_design():
    simulation = False
    tee_fillet = 80

    group =11
    if group == 0:
        
        ### 7.25 GHz ###
        ID = '0'
        f = 7.25e9-0.175e9
        coupling_length = 300
        if simulation:
            prime_length = coupling_length+700
        else:
            prime_length = 0
        down_length = 200
        coupling_space = 70
        tee_pos = [-2500, 500]
        right = True
        orientation = '90'
        res_gap = '30um'
        short_pos = [-1000, tee_pos[1]-coupling_length+200]

    if group ==1:
        ### 7 GHz ###
        ID = '1'
        f = 7e9-0.12e9
        coupling_length = 300
        if simulation:
            prime_length = coupling_length+700
        else:
            prime_length = 0
        down_length = 150
        coupling_space = 70
        tee_pos = [-2500, 2500]
        right = True
        orientation = '90'
        res_gap = '20um'
        short_pos = [-900, tee_pos[1]-coupling_length+150]

    if group ==2:
        ### 6.75 GHz ###
        ID = '2'
        f = 6.75e9-0.08e9
        coupling_length = 300
        if simulation:
            prime_length = coupling_length+700
        else:
            prime_length = 0
        down_length = 150
        coupling_space = 48
        tee_pos = [2500, -2500]
        right = False
        orientation = '-90'
        res_gap = '10um'
        short_pos = [950, tee_pos[1]]

    if group ==3: 
        ### 6.5 GHz ###
        ID = '3'
        f = 6.5e9-0.17e9
        coupling_length = 200
        if simulation:
            prime_length = coupling_length+700
        else:
            prime_length = 0
        down_length = 150
        coupling_space = 50
        tee_pos = [2500, -500]
        right = False
        orientation = '-90'
        res_gap = '30um'
        short_pos = [900, tee_pos[1]]

    if group==4:
        ### 6 GHz ###
        ID = '4'
        f = 6e9-0.11e9
        coupling_length = 200
        if simulation:
            prime_length = coupling_length+700
        else:
            prime_length = 0
        down_length = 150
        coupling_space = 35
        tee_pos = [0, 1500]
        right = False
        orientation = '-90'
        res_gap = '20um'
        short_pos = [-1950, tee_pos[1]-coupling_length]

    if group==5:
        ### 5.75 GHz ###
        ID = '5'
        f = 5.75e9-0.081e9
        coupling_length = 200
        if simulation:
            prime_length = coupling_length+700
        else:
            prime_length = 0
        down_length = 150
        coupling_space = 32
        tee_pos = [0, 500]
        right = True
        orientation = '90'
        res_gap = '10um'
        short_pos = [1550, tee_pos[1]-coupling_length]

    if group==6:
        ### 5.5 GHz ###
        ID = '6'
        f = 5.5e9-0.14e9
        coupling_length = 300
        if simulation:
            prime_length = coupling_length+700
        else:
            prime_length = 0
        down_length = 150
        coupling_space = 50
        tee_pos = [0, -500]
        right = False
        orientation = '-90'
        res_gap = '30um'
        short_pos = [-1800, tee_pos[1]]

    if group==7:
        ### 5.4 GHz ###
        ID = '7'
        f = 5.4e9-0.105e9
        coupling_length = 300
        if simulation:
            prime_length = coupling_length+700
        else:
            prime_length = 0
        down_length = 150
        coupling_space = 53
        tee_pos = [0, -1500]
        right = False
        orientation = '-90'
        res_gap = '20um'
        short_pos = [-2000, tee_pos[1]]

    if group==8:

        ### 5.3 GHz ###
        ID = '8'
        f = 5.3e9-0.065e9
        coupling_length = 200
        if simulation:
            prime_length = coupling_length+700
        else:
            prime_length = 0
        down_length = 150
        coupling_space = 35
        tee_pos = [0, -1500]
        right = True
        orientation = '90'
        res_gap = '10um'
        short_pos = [1950, tee_pos[1]-coupling_length]

    if group==9:
        ### 5 GHz ###
        ID = '9'
        f = 5e9-0.12e9
        coupling_length = 200
        if simulation:
            prime_length = coupling_length+700
        else:
            prime_length = 0
        down_length = 150
        coupling_space = 40
        tee_pos = [0, -2500]
        right = False
        orientation = '-90'
        res_gap = '30um'
        short_pos = [-1950, tee_pos[1]-coupling_length]

    if group==10:
        ### 4.75 GHz ###
        ID = '10'
        f = 4.75e9-0.08e9
        coupling_length = 200
        if simulation:
            prime_length = coupling_length+700
        else:
            prime_length = 0
        down_length = 150
        coupling_space = 30
        tee_pos = [0, 1500]
        right = True
        orientation = '90'
        res_gap = '20um'
        short_pos = [2000, tee_pos[1]-coupling_length]
    
    if group==11:
        ### 4.5 GHz ###
        ID = '11'
        f = 4.5e9-0.05e9
        coupling_length = 300
        if simulation:
            prime_length = coupling_length+700
        else:
            prime_length = 0
        down_length = 200
        coupling_space = 33
        tee_pos = [0, 2500]
        right = True
        orientation = '90'
        res_gap = '10um'
        short_pos = [2250, tee_pos[1]-coupling_length]

    resonator(ID, f, tee_pos, coupling_length, prime_length, coupling_space, short_pos, res_gap=res_gap, down_length = down_length, fillet = tee_fillet, right = right, orientation=orientation)

    gui.rebuild()
    gui.autoscale() 

In [ ]:
render_qiskit_metal_design()

In [ ]:
import names as n
import numpy as np
import parameter_targets as pt

from qdesignoptimizer.design_analysis_types import MiniStudy
from qdesignoptimizer.sim_capacitance_matrix import (
    CapacitanceMatrixStudy,
    ModeDecayIntoChargeLineStudy,
)
from qdesignoptimizer.utils.names_design_variables import junction_setup
from qdesignoptimizer.utils.names_parameters import FREQ, param

import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qdesignoptimizer.design_analysis_types import MeshingMap

from typing import List

import names as n
import numpy as np

from qdesignoptimizer.design_analysis_types import OptTarget
from qdesignoptimizer.utils.optimization_targets import (
    get_opt_targets_qb_res_transmission,
)

In [ ]:
from qdesignoptimizer.utils.utils import close_ansys

In [ ]:
close_ansys()

In [ ]:
ID = '11'

cap_study = CapacitanceMatrixStudy(
        qiskit_component_names=['res'+ID, 'tee'+ID],
        open_pins=[
            ('res'+ID, 'end'),
            ('tee'+ID, "prime_start"),
            ('tee'+ID, "prime_end"),
        ],
        mode_freq_GHz=7.25 * 1e-9,
        nbr_passes=10,
        percent_error=0.01,
    )

In [ ]:
cap_study.simulate_capacitance_matrix(design)

In [ ]:
def qc(imp, cap, omega):
    return np.pi / (imp**2 * omega**2 * cap**2)

In [ ]:
freq= 7.25e9
2*freq/qc(60, 0.89915e-15, 2*np.pi*freq)/1e3

In [ ]:
capacitance_results = {'res0': {'cap': 0.89915, 'freq': 7.25e9-0.175e9},
                       'res1':  {'cap': 0.87076, 'freq': 7e9-0.12e9}, 
                       'res2':  {'cap': 1.22622, 'freq': 6.75e9-0.08e9},
                       'res3':  {'cap': 0.84472, 'freq': 6.5e9-0.17e9}, 
                       'res4':  {'cap': 1.09915, 'freq': 6e9-0.11e9},
                       'res5':  {'cap': 1.20539, 'freq': 5.75e9-0.081e9},
                       'res6':  {'cap': 1.27141, 'freq': 5.5e9-0.14e9},
                       'res7':  {'cap': 1.18791, 'freq': 5.4e9-0.105e9},
                       'res8':  {'cap': 1.09906, 'freq': 5.3e9-0.065e9},
                       'res9':  {'cap': 1.04008, 'freq': 5e9-0.12e9}, 
                       'res10':  {'cap': 1.28865, 'freq': 4.75e9-0.08e9}, 
                       'res11':  {'cap': 1.75526, 'freq': 4.5e9-0.05e9}}

In [ ]:
freqs = []
kappas = []
for i in range(len(capacitance_results)):
    cap = capacitance_results['res'+str(i)]['cap']
    freq = capacitance_results['res'+str(i)]['freq']
    kappa = 2*freq/qc(60, cap*1e-15, 2*np.pi*freq)/1e3
    capacitance_results['res'+str(i)]['kappa'] = kappa
    kappas.append(kappa)
    freqs.append(freq)

kappas = np.array(kappas)
freqs = np.array(freqs)

In [ ]:
kappa_measured = np.array([38447, 23705, 36409, 21694, 22233, 19941, 32815, 14795, 18998, 12448, 22766, 22881])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.close()
plt.title('TD Nb resonators chip 1')
plt.plot(freqs/1e9, kappas, 'o-', label='simulated')
plt.plot(freqs/1e9, kappa_measured/1e3, 'o-', label='measured')
plt.ylim(0,50)
plt.ylabel('KAPPA (kHz)')
plt.xlabel('FREQ (GHz)')
plt.legend()
plt.show()

In [ ]:
38447, 23705, 36409, 21694